In [ ]:
# load binary file
import numpy as np
import time
import suite2p, register, dcnv, celldetect
import imp
imp.reload(suite2p)
imp.reload(register)
imp.reload(celldetect)

def tic():
    return time.time()
def toc(i0):
    return time.time() - i0

ops = {
    'diameter': 12,
    'tau': 2.,
    'fs': 2.5,
    'data_path': ['C:/DATA/'],
    #'data_path': 'C:\Drive\suite2python\data',
    'subfolders': ['5', '6'],
    'num_workers': 0,
    'reg_tif':True,
    'max_iterations':10,
      }


# copy tiff to a binary
i0 = tic()
ops1 = suite2p.main(ops)
print(toc(i0))
#ops1 = register.tiff_to_binary(ops)

In [ ]:
ops1[0]['outer_neuropil_radius'] is np.inf

In [ ]:
ops1[0]['outer_neuropil_radius']

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
%matplotlib inline
plt.figure(figsize=(14,14))
plt.imshow(ops1[0]['meanImg'])
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(ops1[0]['yoff'])
plt.show()

In [ ]:
import imp
imp.reload(register)

i0 = tic()
#ops1[0]['batch_size'] = 200
ops1[0]['num_workers'] = 0

# register tiff
ops1 = register.register_binary(ops1)

print(toc(i0))

In [ ]:
import imp
imp.reload(celldetect)

ops = ops1[0]
U,sdmov      = celldetect.getSVDdata(ops)
S, StU , StS = celldetect.getStU(ops, U)
print(U.shape)

In [ ]:
ops['diameter'] = 12
# get ROIs
ops, stat, cell_masks, neuropil_masks, mPix, mLam = celldetect.sourcery(ops, U, S, StU, StS)
# extract fluorescence and neuropil
F, Fneu = celldetect.extractF(ops, stat, cell_masks, neuropil_masks, mPix, mLam)
# deconvolve fluorescence
spks = dcnv.oasis(F - ops['neucoeff'] * Fneu, ops)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
%matplotlib inline
import scipy.stats as stats

dF = F - 0.7*Fneu
dF = stats.zscore(dF, axis=1)

sk = stats.skew(dF, axis=1)
sd = np.std(dF, axis=1)
print(np.mean(sk[sd>0]))

plt.figure(figsize=(18,18))
plt.imshow(dF[:,:], vmax=5, aspect='auto')
plt.show()
